<h2 style='color:blue' align='center'>Implementation grandient descent in python</h2>

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import random

##### Load the dataset in pandas dataframe

In [4]:
df = pd.read_csv("homeprices_banglore.csv")
df.sample(5)

,area,bedrooms,price
13,1100,2,40.00
0,1056,2,39.07
7,3300,4,155.00
14,2250,3,101.00
5,1170,2,38.00


##### Preprocessing/Scaling: Since our columns are on different sacle it is important to perform scaling on them

In [5]:
from sklearn import preprocessing
sx = preprocessing.MinMaxScaler()
sy = preprocessing.MinMaxScaler()

scaled_X = sx.fit_transform(df.drop('price',axis='columns'))
scaled_y = sy.fit_transform(df['price'].values.reshape(df.shape[0],1))

scaled_X

array([[0.08827586, 0.25      ],
       [0.62068966, 0.75      ],
       [0.22068966, 0.5       ],
       [0.24862069, 0.5       ],
       [0.13793103, 0.25      ],
       [0.12758621, 0.25      ],
       [0.6662069 , 0.75      ],
       [0.86206897, 0.75      ],
       [0.17586207, 0.5       ],
       [1.        , 1.        ],
       [0.34482759, 0.5       ],
       [0.68448276, 0.75      ],
       [0.06896552, 0.25      ],
       [0.10344828, 0.25      ],
       [0.5       , 0.5       ],
       [0.12931034, 0.25      ],
       [0.13103448, 0.5       ],
       [0.25517241, 0.5       ],
       [0.67931034, 0.5       ],
       [0.        , 0.        ]])

In [6]:
scaled_y

array([[0.05237037],
       [0.65185185],
       [0.22222222],
       [0.31851852],
       [0.14074074],
       [0.04444444],
       [0.76296296],
       [0.91111111],
       [0.13333333],
       [1.        ],
       [0.37037037],
       [0.8       ],
       [0.04444444],
       [0.05925926],
       [0.51111111],
       [0.07407407],
       [0.11851852],
       [0.20740741],
       [0.51851852],
       [0.        ]])

##### We should convert target column (i.e. price) into one dimensional array. It has become 2D due to scaling that we did above but now we should change to 1D

In [7]:
scaled_y.reshape(20,)

array([0.05237037, 0.65185185, 0.22222222, 0.31851852, 0.14074074,
       0.04444444, 0.76296296, 0.91111111, 0.13333333, 1.        ,
       0.37037037, 0.8       , 0.04444444, 0.05925926, 0.51111111,
       0.07407407, 0.11851852, 0.20740741, 0.51851852, 0.        ])

In [16]:
class Gradient_Descent:
    def __init__(self, learning_rate = 0.01 , epochs = 500):
        self.w             = 0
        self.b             = 0
        self.cost          = 0
        self.cost_list     = []
        self.epoch_list    = []
        self.learning_rate = learning_rate
        self.epochs        = epochs 
    
    def batch_gradient_descent(self,X, y_true):
        number_of_features = X.shape[1]
        # numpy array with 1 row and columns equal to number of features.
        self.w = np.ones(shape=(number_of_features)) 
        self.b = 0
        total_samples = X.shape[0] # number of rows in X
    
        for i in range(self.epochs):        
            y_predicted = np.dot(self.w, X.T) + self.b

            w_grad = -(2/total_samples)*(X.T.dot(y_true-y_predicted))
            b_grad = -(2/total_samples)*np.sum(y_true-y_predicted)
        
            self.w = self.w - self.learning_rate * w_grad
            self.b = self.b - self.learning_rate * b_grad
        
            self.cost = np.mean(np.square(y_true-y_predicted)) # MSE (Mean Squared Error)
        
            if i%10==0:
                self.cost_list.append(self.cost)
                self.epoch_list.append(i)
        return self.w, self.b, self.cost, self.cost_list, self.epoch_list

    def stochastic_gradient_descent(self,X, y_true):
        number_of_features = X.shape[1]
        # numpy array with 1 row and columns equal to number of features. 
        self.w = np.ones(shape=(number_of_features)) 
        self.b = 0
        total_samples = X.shape[0]

        for i in range(self.epochs):    
            random_index = random.randint(0,total_samples-1) # random index from total samples
            sample_x = X[random_index]
            sample_y = y_true[random_index]
        
            y_predicted = np.dot(self.w, sample_x.T) + self.b
    
            w_grad = -(2/total_samples)*(sample_x.T.dot(sample_y-y_predicted))
            b_grad = -(2/total_samples)*(sample_y-y_predicted)
        
            self.w = self.w - self.learning_rate * w_grad
            self.b = self.b - self.learning_rate * b_grad
        
            self.cost = np.square(sample_y-y_predicted)
        
            if i%100==0: # at every 100th iteration record the cost and epoch value
                self.cost_list.append(self.cost)
                self.epoch_list.append(i)
        
        return self.w, self.b, self.cost, self.cost_list, self.epoch_list
    
    def Mini_BCD(self,X, y_true, size_of_batch = 10):
        number_of_features = X.shape[1]
        # numpy array with 1 row and columns equal to number of features.
        self.w = np.ones(shape=(number_of_features)) 
        self.b = 0
        total_samples = X.shape[0]

        for i in range(self.epochs):    
            random_index = [random.randrange(0, total_samples, 1) for i in range(size_of_batch)] # random index from total samples
            sample_x = X[random_index]
            sample_y = y_true[random_index]
        
            y_predicted = np.dot(self.w, sample_x.T) + self.b
    
            w_grad = -(2/total_samples)*(sample_x.T.dot(sample_y-y_predicted))
            b_grad = -(2/total_samples)*np.sum(sample_y-y_predicted)
        
            self.w = self.w - self.learning_rate * w_grad
            self.b = self.b - self.learning_rate * b_grad
        
            self.cost = np.mean(np.square(sample_y-y_predicted))
        
            if i%100==0: # at every 100th iteration record the cost and epoch value
                self.cost_list.append(self.cost)
                self.epoch_list.append(i)
        
        return self.w, self.b, self.cost, self.cost_list, self.epoch_list
    
    def show_cost_vs_epoch(self):
        plt.xlabel("epoch")
        plt.ylabel("cost")
        plt.plot(self.epoch_list,self.cost_list)


In [8]:
def predict(area,bedrooms,w,b):
    scaled_X = sx.transform([[area, bedrooms]])[0]
    # here w1 = w[0] , w2 = w[1], w3 = w[2] and bias is b
    # equation for price is w1*area + w2*bedrooms + w3*age + bias
    # scaled_X[0] is area
    # scaled_X[1] is bedrooms
    # scaled_X[2] is age
    scaled_price = w[0] * scaled_X[0] + w[1] * scaled_X[1] + b
    # once we get price prediction we need to to rescal it back to original value
    # also since it returns 2D array, to get single value we need to do value[0][0]
    return sy.inverse_transform([[scaled_price]])[0][0]

predict(2600,4,w,b)

128.45484403267596